**FYI: THIS IS BEING WORKED ON. Please be careful before refering.**

Questions:

What is the picture of digital connectivity and engagement in 2020?

What is the effect of the COVID-19 pandemic on online and distance learning, and how might this also evolve in the future?

How does student engagement with different types of education technology change over the course of the pandemic?

How does student engagement with online learning platforms relate to different geography? Demographic context (e.g., race/ethnicity, ESL, learning disability)? Learning context? Socioeconomic status?

Do certain state interventions, practices or policies (e.g., stimulus, reopening, eviction moratorium) correlate with the increase or decrease online engagement?

In [1]:
#johnniekips@mail.com
#john-tech codes
import pandas as pd #for data processing
import numpy as np #for linear algebra 
import seaborn as sns 
pal = sns.color_palette()
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
from plotnine import*
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from sklearn import preprocessing
import glob
from geopy.geocoders import Nominatim
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
from keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
import re
import datetime as dt
import missingno as msno
from tqdm.notebook import tqdm
tqdm.pandas()
import pathlib 
PATH = '/kaggle/input/learnplatform-covid19-impact-on-digital-learning'
#listing all the files available under this directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
readme = open('../input/learnplatform-covid19-impact-on-digital-learning/README.md')
outline = readme.readlines()

In [1]:
district_info=pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
print(district_info.shape)
district_info.head()

In [1]:
products_info=pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
print(products_info.shape)
products_info.head()

In [1]:
engagement_info=pd.DataFrame()
address = glob.glob('../input/learnplatform-covid19-impact-on-digital-learning/engagement_data/*.csv')
count=0
for i in address:
    with open(i, "rb") as data_of_files:
        data=pd.read_csv(data_of_files)
        data['district_id'] = (i.split('/')[4]).split('.')[0]
        engagement_info=pd.concat([engagement_info,data], axis=0)
        count=count+1
        if count==233:
            break  
print(engagement_info.shape)            
engagement_info.head()

In [1]:
district_info.columns

In [1]:
products_info.columns

In [1]:
engagement_info.columns

In [1]:
district_info.info()

In [1]:
products_info.info()

In [1]:
engagement_info.info()

In [1]:
district_null = pd.DataFrame(district_info.isnull().sum()/len(district_info)).reset_index().sort_values(by=0)
plt.figure(figsize=(8,6))
plt.bar(district_null['index'],district_null[0],alpha=0.75)
plt.xticks(rotation=90)
plt.rc('ytick', labelsize=10)
plt.title('Percetage of missing values in district data')

In [1]:
product_null = pd.DataFrame(products_info.isnull().sum()/len(products_info)).reset_index().sort_values(by=0)
plt.figure(figsize=(8,6))
plt.bar(product_null['index'],product_null[0],alpha=0.75)
plt.xticks(rotation=90)
plt.rc('ytick', labelsize=10)
plt.title('Percetage of missing values in products data')

In [1]:
engagement_null = pd.DataFrame(engagement_info.isnull().sum()/len(engagement_info)).reset_index().sort_values(by=0)
plt.figure(figsize=(8,6))
plt.bar(engagement_null['index'],engagement_null[0],alpha=0.75)
plt.xticks(rotation=90)
plt.rc('ytick', labelsize=10)
plt.title('Percetage of missing values in engagement data')

In [1]:
locations=pd.DataFrame({"Name":district_info['state'].unique()})
all_loc=pd.DataFrame(district_info['state'].value_counts().reset_index())
all_loc.columns=['Name','count']
full_loc=all_loc.merge(locations,on='Name',how="left").dropna()

geolocator=Nominatim(user_agent="app")
lat=[]
lon=[]
for location in locations['Name']:
    location = geolocator.geocode(location)    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)
        
locations['lat']=lat
locations['lon']=lon
display(locations)
all_loc=pd.DataFrame(district_info['state'].value_counts().reset_index())
all_loc.columns=['Name','count']
full_loc=all_loc.merge(locations,on='Name',how="left").dropna()

In [1]:
def generateBaseMap(default_location=[37.0902, -95.7129], default_zoom_start=4):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

basemap=generateBaseMap()

In [1]:
HeatMap(full_loc[['lat','lon','count']],zoom=20,radius=20).add_to(basemap)
display(basemap)

In [1]:
FastMarkerCluster(full_loc[['lat','lon','count']],zoom=20,radius=20).add_to(basemap)
display(basemap)

In [1]:
plt.figure(figsize=(10,10))
sns.countplot(district_info.state)
plt.xticks(rotation=90)

In [1]:
district_info["state"].value_counts().head(10).plot(kind = 'pie', autopct='%1.1f%%',figsize=(10, 10), startangle=0).legend()

In [1]:
district_info["state"].value_counts().tail(10).plot(kind = 'pie', autopct='%1.1f%%', figsize=(10, 10), startangle=0).legend()

In [1]:
plt.figure(figsize=(10,10))
sns.countplot(district_info.locale)
plt.xticks(rotation=90,color='red')

In [1]:
district_info["locale"].value_counts().head().plot(kind = 'pie', autopct='%1.1f%%', figsize=(10, 10), startangle=0).legend()

In [1]:
map_plot = district_info.dropna(subset=['state'])
blc_hisp_count = map_plot.groupby(['locale','pct_black/hispanic'],as_index=False)['district_id'].count()
pct_blk_hisp = {'[0, 0.2[':'0-20%', '[0.2, 0.4[':'20-40%' ,'[0.4, 0.6[':'40-60%', '[0.8, 1[':'80-100%', '[0.6, 0.8[':'60-80%'}
blc_hisp_count['Percentage of black and hipanic population']=blc_hisp_count['pct_black/hispanic'].map(pct_blk_hisp)
blc_hisp_count = blc_hisp_count.rename(columns={'district_id':'Number of districts'})
city_b = blc_hisp_count[blc_hisp_count.locale=='City']
town_b = blc_hisp_count[blc_hisp_count.locale=='Town']
suburb_b = blc_hisp_count[blc_hisp_count.locale=='Rural']
rural_b = blc_hisp_count[blc_hisp_count.locale=='Suburb']
from plotly.subplots import make_subplots
fig3 = make_subplots(rows=2, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'},{'type':'domain'}]])

fig3.add_trace(go.Pie(labels=suburb_b['Percentage of black and hipanic population'], values=suburb_b['Number of districts'], name="Suburb"),
              1, 1)
fig3.add_trace(go.Pie(labels=city_b['Percentage of black and hipanic population'], values=city_b['Number of districts'], name="City"),
              1, 2)
fig3.add_trace(go.Pie(labels=rural_b['Percentage of black and hipanic population'], values=rural_b['Number of districts'], name="Rural"),
              2, 1)
fig3.add_trace(go.Pie(labels=town_b['Percentage of black and hipanic population'], values=town_b['Number of districts'], name="Town"),
              2, 2)
# Use `hole` to create a donut-like pie chart
fig3.update_traces(hole=.4, hoverinfo="label+percent+name")

fig3.update_layout(
    title_text="Distribution of pct_black/hispanic in different localities",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Suburb', x=0.17, y=.8, font_size=20, showarrow=False),
                 dict(text='City', x=0.8, y=.8, font_size=20, showarrow=False),
                 dict(text='Rural', x=0.18, y=0.175, font_size=20, showarrow=False),
                dict(text='Town', x=0.82, y=0.175, font_size=20, showarrow=False)], width=900, height=800)

fig3.show()